# Load the Data

In [26]:
from sklearn.datasets import load_iris
import torch

In [27]:
data = load_iris()

In [28]:
x_data = torch.tensor(data.data, dtype=torch.float32)
y_data = torch.tensor(data.target, dtype=torch.long)
print(f"x_data shape: {x_data.shape}")
print(f"y_data shape: {y_data.shape}")

x_data shape: torch.Size([150, 4])
y_data shape: torch.Size([150])


In [29]:
x_data

tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000],
        [5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.4000, 1.4000, 0.3000],
        [5.0000, 3.4000, 1.5000, 0.2000],
        [4.4000, 2.9000, 1.4000, 0.2000],
        [4.9000, 3.1000, 1.5000, 0.1000],
        [5.4000, 3.7000, 1.5000, 0.2000],
        [4.8000, 3.4000, 1.6000, 0.2000],
        [4.8000, 3.0000, 1.4000, 0.1000],
        [4.3000, 3.0000, 1.1000, 0.1000],
        [5.8000, 4.0000, 1.2000, 0.2000],
        [5.7000, 4.4000, 1.5000, 0.4000],
        [5.4000, 3.9000, 1.3000, 0.4000],
        [5.1000, 3.5000, 1.4000, 0.3000],
        [5.7000, 3.8000, 1.7000, 0.3000],
        [5.1000, 3.8000, 1.5000, 0.3000],
        [5.4000, 3.4000, 1.7000, 0.2000],
        [5.1000, 3.7000, 1.5000, 0.4000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [5.1000, 3.3000, 1.7000, 0

In [30]:
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Model Definition

In [31]:
class IrisClassifier(nn.Module):
    def __init__(self):
        super(IrisClassifier, self).__init__()
        self.fc1 = nn.Linear(4, 16)   # 4 input features, 16 neurons in the hidden layer
        self.fc2 = nn.Linear(16, 8)   # 16 input neurons, 8 neurons in the next layer
        self.fc3 = nn.Linear(8, 3)    # 8 input neurons, 3 output neurons (for 3 species)
        self.relu = nn.ReLU()         # Activation function

    def forward(self, x):
        x_1 = self.relu(self.fc1(x))
        x_3 = self.relu(self.fc2(x_1))
        x_4 = self.fc3(x_3)               # No activation on the output layer, as we will use CrossEntropyLoss
        return x_4

model = IrisClassifier()
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train Loop

In [32]:
epochs = 50
for epoch in range(epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()  # To prevent accumulation of gradients from all epochs. (Reset Gradients)
    loss.backward()  # Backward Propogation of Loss. (Calculate Gradients)
    optimizer.step()  # Updates the model's parameters according to the calculated gradients.

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/50], Loss: 1.0156
Epoch [20/50], Loss: 0.8361
Epoch [30/50], Loss: 0.6705
Epoch [40/50], Loss: 0.5302
Epoch [50/50], Loss: 0.3056


# Test Loop

In [33]:
model.eval()
with torch.no_grad():
    test_outputs = model(x_test)
    _, predicted = torch.max(test_outputs, 1)
    print(predicted)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f"Accuracy: {accuracy*100:.2f}%")

tensor([1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0])
Accuracy: 96.67%
